# 作业 - Bert (Extractive Question Answering)

> 镜像自李宏毅老师 2023 年机器学习春的作业 7，原链接：[ML2023_HW7_Question_Answering](https://colab.research.google.com/drive/1m0fQjJfkK9vAovxPj9Nd3-hQuxezB2w1)
>
> 延伸文章：[22a. 微调 LLM：实现抽取式问答](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/22a.%20微调%20LLM：实现抽取式问答.md)

请根据**提示**完成相应的代码修改，从而达到学习的目的。

[作业 PPT](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/PDF/HW_BERT_Question_Answering.pdf) | [Kaggle 加入链接](https://www.kaggle.com/t/e001cad568dc4d77b6a5e762172f44d6)

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/21-bert) | [Colab](https://colab.research.google.com/drive/1zHR2Cztmo49j3yrdT3GgkzNcAcHd7M0M?usp=sharing)


## 作业提示

> 如果理解了每个提示到底需要做什么，那么恭喜你已经完成了学习。
>
> 搜索代码文件中的 TODO 可以快速索引到需要修改的地方。

- **Simple baseline**
  - 直接运行当前代码文件
- **Medium baseline**
  - 使用学习率调度器：线性衰减，warmup 等，选择一个即可
- **Strong baseline**
  - 修改模型的预处理过程 (TODO: Preprocessing)
  - 尝试替换为其他预训练模型，在 [Hugging Face](https://huggingface.co/models?language=zh&sort=trending) 中进行选择
- **Boss baseline**
  - 修改后处理部分 (TODO: Postprocessing)
  - 尝试梯度累积

## Kaggle 加入“比赛”

访问 [Kaggle 加入链接](https://www.kaggle.com/t/e001cad568dc4d77b6a5e762172f44d6)，点击右上角的 `Late Submission`，然后点击 `I Understand and Accept` 同意：

![image-20241110191905666](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20241110191905666.png?raw=1)

样例代码运行后会生成文件：`result.csv`，如果想要提交查看分数，再次点击 `Late Submission`，上传后点击提交：

![image-20241110192134533](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20241110192134533.png?raw=1)

你可以在 `Submissions` 中看到自己的提交成绩：

![image-20241110192753327](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20241110192753327.png?raw=1)

> [!note]
>
> 在评估效果时最好仅参考 Public Score，Private Score 是测试集上的分数，因为比赛已经过了Deadline，所以显示了出来，比赛进行的过程中这个分数是隐藏的。

点击 `Leaderboard` 你将能够看到曾经参与者的成绩：

![image-20241110192649655](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20241110192649655.png?raw=1)


## 任务目标：中文抽取式问答

学习如何使用 transformers 微调预训练模型，以完成下游抽取式问答任务。

- **任务描述**: 通过 fine-tune BERT 模型，使其能够从给定的段落中抽取出问题的具体答案。
  - **目标**: 模型将预测答案的开始（Start）和结束（End）位置，并从文本中提取该片段作为答案。
    
    ![image-20240920002112364](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/e6a56c94334632bae17b286f42264de4.png?raw=1)
    
  - **输入**: 段落 + 问题
  - **输出**: 答案
 
- **TODO**
  - 微调预训练的中文 BERT 模型
  - 调整超参数（如 `doc_stride`）
  - 应用线性学习率衰减策略
  - 尝试其他预训练模型
  - 改进数据的预处理（preprocessing）
  - 改进模型的后处理（postprocessing）

- **训练技巧**
  - 自动混合精度（Automatic Mixed Precision）
  - 梯度累积（Gradient Accumulation）
  - 集成方法（Ensemble）

- **预计训练时间**（在 Tesla T4 上启用自动混合精度时的运行时间）：
  - Simple baseline：8 分钟
  - Medium baseline：8 分钟
  - Strong baseline：25 分钟
  - Boss baseline：2 小时

## 性能指标（EM）

- **准确率 (Exact Match)**: 该指标用于衡量模型的预测答案与真实答案**完全一致**的比例。



## 前置准备

### 下载数据集

#### Kaggle

> To use the Kaggle API, sign up for a Kaggle account at [https://www.kaggle.com](https://www.kaggle.com/). Then go to the 'Account' tab of your user profile (`https://www.kaggle.com/<username>/account`) and select 'Create API Token'. This will trigger the download of `kaggle.json`, a file containing your API credentials. Place this file in the location `~/.kaggle/kaggle.json` (on Windows in the location `C:\Users\<Windows-username>\.kaggle\kaggle.json` - you can check the exact location, sans drive, with `echo %HOMEPATH%`). You can define a shell environment variable `KAGGLE_CONFIG_DIR` to change this location to `$KAGGLE_CONFIG_DIR/kaggle.json` (on Windows it will be `%KAGGLE_CONFIG_DIR%\kaggle.json`).
>
> -\- [Official Kaggle API](https://github.com/Kaggle/kaggle-api)

替换\<username\>为你自己的用户名，`https://www.kaggle.com/<username>/account`，然后点击 `Create New API Token`，将下载下来的文件放去应该放的位置：

- Mac 和 Linux 放在 `~/.kaggle`
- Windows 放在 `C:\Users\<Windows-username>\.kaggle`

```bash
pip install kaggle
# 你需要先在 Kaggle -> Account -> Create New API Token 中下载 kaggle.json
# mv kaggle.json ~/.kaggle/kaggle.json
kaggle competitions download -c ml2023spring-hw7
unzip ml2023spring-hw7.zip   
```


In [ ]:
# 或者通过以下命令进行下载
!wget https://github.com/Hoper-J/HUNG-YI_LEE_Machine-Learning_Homework/raw/refs/heads/master/HW07/ml2023spring-hw7.zip
!unzip ml2023spring-hw7.zip

### 安装库 

In [ ]:
!pip install transformers
!pip install accelerate

## 开始微调


### 导入包



In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# 为了可重复性，固定随机数种子
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(42)

### 加载模型和分词器


In [ ]:
from transformers import (
  AutoTokenizer,
  AutoModelForQuestionAnswering,
)

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# 你可以忽略警告消息（它弹出是因为新的 QA 预测头是随机初始化的）

## 数据部分

> 使用两个繁体中文阅读理解数据集：[DRCD](https://github.com/DRCKnowledgeTeam/DRCD) 和 [ODSQA](https://github.com/Chia-Hsuan-Lee/ODSQA)。

- **训练集（DRCD + DRCD-backtrans）**：包含 15,329 个段落和 26,918 个问题。一个段落可能对应多个问题。
- **开发集（DRCD + DRCD-backtrans）**：包含 1,255 个段落和 2,863 个问题。用于验证。
- **测试集（DRCD + ODSQA）**：包含 1,606 个段落和 3,504 个问题。测试集的段落没有提供答案，需要模型进行预测。

所有数据集的格式相同：

- `id`：问题编号
- `paragraph_id`：段落编号
- `question_text`：问题文本
- `answer_text`：答案文本
- `answer_start`：答案在段落中的起始字符位置
- `answer_end`：答案在段落中的结束字符位置

![数据集格式](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/4215768313590de87aab01adcad78c90.png?raw=1)

### 读取数据

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

### 分词处理

In [ ]:
# 分别对问题和段落进行分词
# 「add_special_tokens」设置为 False，因为在数据集的 __getitem__ 中，当合并分词后的问题和段落时会添加特殊标记

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# 你可以忽略警告消息，因为在传递给模型之前，将在数据集的 __getitem__ 中进一步处理分词后的序列

### 数据集（Medium / Strong 相关）


In [ ]:
class QA_Dataset(Dataset):
    """
    自定义的问答数据集类，用于处理问答任务的数据。

    参数：
    - split (str): 数据集的类型，'train'、'dev' 或 'test'。
    - questions (list): 问题列表，每个元素是一个字典，包含问题的详细信息。
    - tokenized_questions (BatchEncoding): 分词后的问题，由 tokenizer 生成。
    - tokenized_paragraphs (BatchEncoding): 分词后的段落列表，由 tokenizer 生成。
    
    属性（即 __init_() 中的 self.xxx）：
    - max_question_len (int): 问题的最大长度（以分词后的 token 数计）。
    - max_paragraph_len (int): 段落的最大长度（以分词后的 token 数计）。
    - doc_stride (int): 段落窗口滑动步长。
    - max_seq_len (int): 输入序列的最大长度。
    """
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 150

        ##### TODO: 更改 doc_stride 的值 #####
        self.doc_stride = 150

        # 输入序列长度 = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        """
        返回数据集中样本的数量。

        返回：
        - (int): 数据集的长度。
        """
        return len(self.questions)

    def __getitem__(self, idx):
        """
        获取数据集中指定索引的样本。

        参数：
        - idx (int): 样本的索引。

        返回：
        - 对于训练集，返回一个输入张量和对应的答案位置：
          (input_ids, token_type_ids, attention_mask, answer_start_token, answer_end_token)
        - 对于验证和测试集，返回包含多个窗口的输入张量列表：
          (input_ids_list, token_type_ids_list, attention_mask_list)
        """
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: 预处理 #####
        # 提示：思考怎么防止模型学习到不应该学习的东西
        if self.split == "train":
            # 将答案在段落文本中的起始/结束位置转换为在分词后段落中的起始/结束位置
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # 通过切片包含答案的段落部分来获得一个窗口
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # 切片问题/段落并添加特殊标记（101：CLS，102：SEP）
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]

            # 将答案在分词后段落中的起始/结束位置转换为窗口中的起始/结束位置
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # 填充序列并获取模型的输入
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # 验证/测试
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []

            # 段落被分割成多个窗口，每个窗口的起始位置由步长 "doc_stride" 分隔
            for i in range(0, len(tokenized_paragraph), self.doc_stride):

                # 切片问题/段落并添加特殊标记（101：CLS，102：SEP）
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]

                # 填充序列并获取模型的输入
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        """
        对输入的序列进行填充，生成统一长度的模型输入。

        参数：
        - input_ids_question (list): 问题部分的输入 ID 列表。
        - input_ids_paragraph (list): 段落部分的输入 ID 列表。

        返回：
        - input_ids (list): 填充后的输入 ID 列表。
        - token_type_ids (list): 区分问题和段落的标记列表。
        - attention_mask (list): 注意力掩码列表，指示哪些位置是有效的输入。
        """
        # 计算需要填充的长度
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # 填充输入序列
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # 构造区分问题和段落的 token_type_ids
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # 构造注意力掩码，有效位置为 1，填充位置为 0
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len

        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

> 这里解释一下QA_Dataset，如果觉得太长，可以只查看重点部分。

1. **初始化（`__init__`）**：
   - `split` 决定数据集的类型（训练、验证或测试）。
   - `questions`, `tokenized_questions`, 和 `tokenized_paragraphs` 是原问题和 tokenized 后的问题和段落。
   - `max_question_len` 和 `max_paragraph_len` 分别设定了问题和段落的最大长度。
   - `self.doc_stride`：段落的窗口滑动步长（决定每个窗口之间的重叠部分）。
     - `Sample code` 中将其设置为 150，和 `max_paragraph_len` 一样，意味着窗口之间完全不重叠。
   - `self.max_seq_len`：定义了整个输入序列的最大长度（包含问题和段落）。

2. **`__getitem__`**：
   - 针对给定的索引 `idx`，获取对应问题和段落数据，返回模型需要的输入。
   - **训练集**：定位答案的起始和结束位置，将包含答案的段落部分截取为一个窗口（中心在答案位置附近）。然后将问题和段落合并为一个输入序列，并进行填充。
   - **验证/测试集**：将段落分成多个窗口，每个窗口之间的步长由 `self.doc_stride` 决定，然后将每个窗口作为模型的输入。验证和测试时不需要答案位置，因此只需生成多个窗口作为输入。

3. **填充（`padding`）**：
   - 输入序列可能比最大序列长度短，填充部分用 0 表示。对于问题部分和段落部分，`token_type_ids` 被用来区分它们（0 表示问题，1 表示段落）。`attention_mask` 用于标记有效的输入部分，防止模型对填充部分进行注意力计算。

#### 重点

- `self.doc_stride` 通过控制窗口之间的滑动步长，确保即使答案位于窗口边缘，模型也能通过多个窗口重叠的方式找到答案。
- **训练阶段**不需要使用 `doc_stride`，因为训练时我们已经知道答案的位置，可以直接截取包含答案的窗口。但在**验证和测试**阶段，由于模型并不知道答案的位置，`doc_stride` 保证每个窗口之间有足够的重叠，避免遗漏答案。
- 所以这里存在一个问题，训练过程中模型可能学习到：答案就在中间这一模式。这是我们在 Strong baseline 中需要解决的。

### 评估函数

In [ ]:
def evaluate(data, output):
    ##### TODO: 后处理 #####
    # 后处理存在一个错误和改进的空间
    # 提示：打开你的预测文件，看看有什么问题

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    for k in range(num_of_windows):
        # 通过选择最可能的起始位置/结束位置来获得答案
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        # 答案的概率计算为 start_prob 和 end_prob 的和
        prob = start_prob + end_prob

        # 如果计算的概率大于之前的窗口，则替换答案
        if prob > max_prob:
            max_prob = prob
            # 将标记转换为字符（例如，[1920, 7032] --> "大 金"）
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

    # 移除答案中的空格（例如，"大 金" --> "大金"）
    return answer.replace(' ','')

### 训练

In [ ]:
from accelerate import Accelerator

# 超参数
num_epoch = 1
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8

#### TODO: 梯度累积（可选）####
# 注意：train_batch_size * gradient_accumulation_steps = 有效批次大小
# 如果 CUDA 内存不足，你可以降低 train_batch_size 并提高 gradient_accumulation_steps
# 文档：https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16

# 数据加载器
# 注意：不要更改 dev_loader / test_loader 的批次大小！
# 虽然批次大小=1，但它实际上是由同一对 QA 的多个窗口组成的批次
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

# 将 "fp16_training" 更改为 True 以支持自动混合精度训练（fp16）
fp16_training = True
if fp16_training:
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()

# 文档：https://huggingface.co/docs/accelerate/
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

model.train()

print("开始训练...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for data in tqdm(train_loader):
        # 将所有数据加载到设备
        data = [i.to(device) for i in data]

        # 模型输入：input_ids, token_type_ids, attention_mask, start_positions, end_positions（注意：只有 "input_ids" 是必需的）
        # 模型输出：start_logits, end_logits, loss（提供 start_positions/end_positions 时返回）
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # 选择最可能的起始位置/结束位置
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # 只有当 start_index 和 end_index 都正确时，预测才正确
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()

        train_loss += output.loss

        accelerator.backward(output.loss)

        step += 1
        optimizer.step()
        optimizer.zero_grad()

        ##### TODO: 应用线性学习率衰减 #####

        # 每经过 logging_step，打印训练损失和准确率
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("评估开发集...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # 只有当答案文本完全匹配时，预测才正确
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# 将模型及其配置文件保存到目录「saved_model」
# 即，在目录「saved_model」下有两个文件：「pytorch_model.bin」和「config.json」
# 可以使用「model = BertForQuestionAnswering.from_pretrained("saved_model")」重新加载保存的模型
print("保存模型...")
model_save_dir = "saved_model"
model.save_pretrained(model_save_dir)

### 测试

In [ ]:
print("评估测试集...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
        # 将答案中的逗号替换为空字符串（因为 csv 以逗号分隔）
        # Kaggle 中的答案也以同样的方式处理
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"完成！结果保存在 {result_file}")

## 拓展

### Question 1

>微调（Finetune）和提示（Prompt）之间存在一些差异。上下文学习（In-context Learning）可以让预训练模型在不进行梯度下降的情况下，通过少量示例给出许多下游任务的正确预测。请回答以下问题：
>
>a. **Encoder-only 模型（如 BERT 系列）如何在抽取式问答任务中确定答案？**
>
>b. **Decoder-only 模型（如 GPT 系列）如何在抽取式问答任务中确定答案？**

### Question 2 (In-context learning)

> 尝试不同的 Prompt 并观察 fine-tuning 和 in-context learning 的区别。
>
> 代码所下载的模型是 [facebook/xglm-1.7B](https://huggingface.co/facebook/xglm-1.7B)，实际上你也可以直接去 GPT 或者其他 AI 平台提问，这里的目的是去调整自己的 prompt，从而使模型不经过微调也能获取到正确答案。
>
> 对于“老手”来说，完全可以跳过这个问题。

#### Prompt 对比（错误对比）

1. **Prompt 示例 1**: "根据文章找出问题的答案：{问题}"。
2. **Prompt 示例 2**: "请阅读文章并回答以下问题：{问题}"。
3. **Prompt 示例 3**: "请根据文章信息回答下列问题：{问题}"。

#### Prompt 对比（正确对比）

> ![image-20240920173255150](../Guide/assets/9805a280ab816c41ec21aee5a223bf9d.png)

- 中英对比
- 不同 prompt 对比

**样例 prompt**:
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章1 內容>
問題：<問題1 敘述>
答案：<答案1>
...
文章：<文章n 內容>
問題：<問題n 敘述>
答案：
```

In [ ]:
import torch
import random
import numpy as np

# 为避免 CUDA_OUT_OF_MEMORY
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# 为了可重复性，固定随机种子
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 你可以尝试不同大小的模型
# 当使用 Colab 或 Kaggle 时，超过 20 亿（2B）参数的模型可能会耗尽内存
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

In [ ]:
# 清理模型输出。如果你尝试不同的提示，可能需要自行修正此函数
def clean_text(text):
    # 注意：当你使用单语模型时，冒号可能会变成全角
    text = text.split("答案:")[-1]
    text = text.split(" ")[0]
    return text

In [ ]:
import random
import json

with open("hw7_in-context-learning-examples.json", "r") as f:
    test = json.load(f)

# K-shot 学习
# 给模型 K 个示例，使其获得更好的准确率
# 注意：(1) 当 K >= 4 时，可能会发生 CUDA_OUT_OF_MEMORY。
#       (2) XGLM 的最大输入长度为 2048
K = 2

question_ids = [qa["id"] for qa in test["questions"]]

with open("in-context-learning-result.txt", "w") as f:
    print("ID,Ground-Truth,Prediction", file = f)
    with torch.no_grad():
        for idx, qa in enumerate(test["questions"]):
            # 你可以尝试不同的提示
            prompt = "請從最後一篇的文章中找出最後一個問題的答案\n"
            exist_question_indexs = [question_ids.index(qa["id"])]

            # K-shot 学习：给模型 K 个带有答案的示例
            for i in range(K):
                question_index = question_ids.index(qa["id"])
                while(question_index in exist_question_indexs):
                    question_index = random.randint(0, len(question_ids) - 1)
                exist_question_indexs.append(question_index)
                paragraph_id = test["questions"][question_index]["paragraph_id"]
                prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
                prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
                prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'

            # 最后一个没有答案的问题
            paragraph_id = qa["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{qa["question_text"]}\n'
            prompt += f'答案：'

            inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
            sample = model.generate(**inputs, max_new_tokens = 20)
            text = tokenizer.decode(sample[0], skip_special_tokens=True)

            # 注意：你可以删除这一行，看看会发生什么
            text = clean_text(text)

            print(prompt)
            print(f'正確答案: {qa["answer_text"]}')
            print(f'模型輸出: {text}')
            print()

            print(f"{idx},{qa['answer_text']},{text}", file = f)